In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [248]:
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 모델 저장소 ID를 지정합니다.
    #repo_id = "mistralai/Mistral-7B-v0.1",  # 모델 저장소 ID를 지정합니다.    
    max_new_tokens=128,  
    temperature=0.01, # 샘플링 온도를 설정합니다. 값이 높을수록 더 다양한 출력을 생성합니다.
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백을 설정합니다.
    streaming=True,  # 스트리밍을 사용합니다.
   
)


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\cowbo\.cache\huggingface\token
Login successful


In [233]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("{country}의 수도에 대해서 한글로 알려줘")

In [234]:
chain = prompt | llm 

In [ ]:
chain.invoke({"country":"대한민국"})


## Runnable

RunnablePassthrough

In [124]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

In [125]:
chain = {"country": RunnablePassthrough()} | prompt | llm 

In [ ]:
chain.invoke("미국")

RunnableParallel

In [255]:
prompt1 = PromptTemplate.from_template("ETF 상품인 {stock}의 운용사를 한글로 알려주고 다른 내용은 생략해 ")
prompt2 = PromptTemplate.from_template("ETF 상품인 {stock}의 년 운용보수를 한글로 알려주고 다른 내용은 생략해")

chain1 = {"stock": RunnablePassthrough()} | prompt1 | llm 
chain2 = {"stock": RunnablePassthrough()} | prompt2 | llm 



In [256]:
chain1.invoke("SCHD")

주세요.

The issuer of SCHD is Schwab U.S. Dividend Equity ETF Trust.</s>

'주세요.\n\nThe issuer of SCHD is Schwab U.S. Dividend Equity ETF Trust.</s>'

In [257]:
chain2.invoke("SCHD")

주세요.

The annual operating expense ratio for the Schwab U.S. Dividend Equity ETF (SCHD) is 0.06%.</s>

'주세요.\n\nThe annual operating expense ratio for the Schwab U.S. Dividend Equity ETF (SCHD) is 0.06%.</s>'

In [258]:
combined_chain = RunnableParallel(a=chain1, b=chain2)


In [259]:
combined_chain.invoke("SCHD")

주세요.주
세
요The. annual
 operating
 expenseThe ratio iss foruer of the S SchwabCHD U. is SchwSab. U. DSivid.end Divid Equendity E EquTFity E (TFS TrustCHD.)</s> is 0.06%.</s>

{'a': '주세요.\n\nThe issuer of SCHD is Schwab U.S. Dividend Equity ETF Trust.</s>',
 'b': '주세요.\n\nThe annual operating expense ratio for the Schwab U.S. Dividend Equity ETF (SCHD) is 0.06%.</s>'}

In [260]:
def combine_text(text):
    return text["a"] + "," + text["b"]

In [261]:
final_chain = combined_chain | {"info" : RunnableLambda(combine_text)} | PromptTemplate.from_template("{info}의 내용을 자연스럽게 연결해줘 \n ") | llm
final_chain.invoke("SCHD")

주주세세요.요
.

The
 annualThe operating issuer expense ratio of for SCH theD Schwab is Schw U.ab US..S D.ividend D Equividityend E EquTF (ityS ECHTFD) Trust is.</s> 0.06%.</s>
The Schwab U.S. Dividend Equity ETF Trust, which is the issuer of SCHD, charges an annual operating expense ratio of 0.06%.</s>

'\nThe Schwab U.S. Dividend Equity ETF Trust, which is the issuer of SCHD, charges an annual operating expense ratio of 0.06%.</s>'